In [1]:
import logging
import sys
from pathlib import Path
from celexta.initialize import default_logging, update_logging
log = default_logging()
update_logging(log_level=logging.DEBUG)
logging.getLogger("matplotlib").setLevel(logging.WARNING)
logging.getLogger("numcodecs").setLevel(logging.WARNING)
from astropy.visualization import ZScaleInterval
from astropy.utils.data import get_pkg_data_filename
from astropy.io import fits
from astropy.wcs import WCS
from astropy.visualization.wcsaxes import WCSAxes
from astropy.visualization.wcsaxes import SphericalCircle
from astropy.visualization import imshow_norm
from astropy.coordinates import SkyCoord
import astropy.units as u
import numpy as np
import matplotlib.pyplot as plt
from PyQt6.QtWidgets import QMainWindow
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from celexta.examples import *
import pyqtgraph as pg
%matplotlib qt

2025-02-25 07:22:06 -    DEBUG - OpenGL.acceleratesupport - [acceleratesupport.py: 20 -   <module>] - OpenGL_accelerate module loaded
2025-02-25 07:22:06 -    DEBUG - OpenGL.arrays.arraydatatype - [arraydatatype.py:334 -   <module>] - Using accelerated ArrayDatatype


# Rotate and align images

## Define all images

In [3]:
# PS1
hdu_ps1 = fits.open("~/Downloads/ps1.fits")[0]
# VT full image
hdu_vt = fits.open("~/Downloads/1storb_R_com.fits")[0]
star1 = SkyCoord("5h36m49","-25:20:20", unit=("hourangle","deg"), frame="icrs")
# LS
hdu_ls = fits.open("~/Desktop/ls_img.fits")[0]
# VT subimage
hdul_qim = fits.open("~/.celexta/cache/svom/sb25020504/qim1b_vt.fits")
hdu_qimb0 = hdul_qim[1]
hdu_qimb1 = hdul_qim[2]
hdu_qimr0 = hdul_qim[3]
hdu_qimr1 = hdul_qim[4]
hdu_qimr0.data = hdu_qimr0.data.T
hdu_qimr1.data = hdu_qimr1.data.T
star2 = SkyCoord(113.4812,32.3908, unit=("deg","deg"), frame="icrs")

## Rotate matplotlib axis

In [3]:

def default_check(hdu, star, interval=ZScaleInterval()):
    wcs = WCS(hdu)


    fig = plt.figure(figsize=(6,6))
        # Original image
    ax = fig.add_subplot(1, 1, 1, projection=wcs)
    ax.set_title("Original")
    imshow_norm(hdu.data, ax=ax, origin="lower", cmap="gray", interval=interval)
    ax.coords.grid(True, color="red", linestyle="dotted", linewidth=1)
    ax.scatter(star.ra.deg, star.dec.deg,
                transform=ax.get_transform("world"),
                facecolor="none",
                edgecolors="C0",
                )
    ax.coords[0].set_axislabel("RA")
    ax.coords[1].set_axislabel("Dec")

default_check(hdu_qimr0, star=star2, interval=None)

2025-02-08 13:44:04 -  WARNING - astropy - [logger.py:235 - _showwarning] - FITSFixedWarning: RADECSYS= 'FK5 ' 
the RADECSYS keyword is deprecated, use RADESYSa.


the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]


In [8]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.wcs import WCS
from astropy.visualization.wcsaxes import WCSAxes
from astropy.visualization import imshow_norm
from scipy.ndimage import rotate
from reproject import reproject_interp

def rotate_wcs_axes(hdu, theta_deg, star, interval=None):
    """Rotates the WCS projection of a Matplotlib Axes without modifying the image pixels."""
    default_check(hdu, star=star, interval=interval)
    hdu = hdu.copy()
    wcs = WCS(hdu.header)
    # Get image dimensions
    original_shape = hdu.data.shape
    h, w = original_shape

    # Convert rotation angle to radians
    theta_rad = np.radians(theta_deg)

    # Compute rotation matrix
    R = np.array([
        [np.cos(theta_rad), -np.sin(theta_rad)],
        [np.sin(theta_rad),  np.cos(theta_rad)]
    ])

    # Apply rotation to the WCS CD matrix
    new_cd_matrix = np.dot(R, wcs.wcs.cd)
    wcs.wcs.cd = new_cd_matrix

    # Compute expanded shape to fit rotated image
    diag = int(np.ceil(np.sqrt(h**2 + w**2)))  # Diagonal size to fit full rotation
    new_shape = (diag, diag)

    # Shift the WCS reference pixel (CRPIX) to keep the image centered
    old_center = np.array([w / 2, h / 2])  # Original center in pixels
    new_center = np.array([diag / 2, diag / 2])  # New center in pixels
    shift = new_center - old_center
    wcs.wcs.crpix += shift

    # Rotate the image data using scipy.ndimage.rotate()
    reprojected_data = rotate(hdu.data, -theta_deg, reshape=True)

    # Reproject onto the new WCS grid (ensures correct alignment)
    # reprojected_data, _ = reproject_interp(hdu, wcs, shape_out=new_shape)

    # Save the rotated image
    hdu.data = reprojected_data
    hdu.header.update(wcs.to_header())

    default_check(hdu, star=star, interval=interval)

# Example usage (Replace "image.fits" with your actual file)
rotate_wcs_axes(
    hdu_ls,
    45,
    star=star2,
    # interval=None,
    interval=ZScaleInterval(),
    )  # Rotates the WCS by 45°

461

In [18]:
from matplotlib.transforms import Affine2D
import mpl_toolkits.axisartist.floating_axes as floating_axes
wcs = WCS(hdu_qimb0)
fig = plt.figure()
plot_extents = 0, hdu_qimb0.header["NAXIS1"], 0, hdu_qimb0.header["NAXIS2"]
transform = Affine2D().rotate_deg(45)
helper = floating_axes.GridHelperCurveLinear(transform, plot_extents)
ax = floating_axes.FloatingSubplot(fig, 111, grid_helper=helper)
ax.imshow(hdu_qimb0.data, vmin=0, vmax=1, origin="lower")#, transform=ax.get_transform("world"))
fig.add_subplot(ax)
fig.canvas.draw()
plt.show()

2025-02-08 13:57:07 -  WARNING - astropy - [logger.py:235 - _showwarning] - FITSFixedWarning: RADECSYS= 'FK5 ' 
the RADECSYS keyword is deprecated, use RADESYSa.


the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]


## Rotate image to north

In [8]:
from reproject import reproject_interp
hdu1 = fits.open(url)[0]
hdu2 = get_vt_img()
array, footprint = reproject_interp(hdu2, hdu1.header)


2025-02-08 07:24:01 -    DEBUG - celexta.examples - [examples.py: 93 - get_vt_img] - Getting VT image
2025-02-08 07:24:01 -    DEBUG - celexta.examples - [examples.py: 93 - get_vt_img] - Getting VT image
2025-02-08 07:24:01 -  WARNING - astropy - [logger.py:235 - _showwarning] - FITSFixedWarning: TSTART = / null value 
a floating-point value was expected.
2025-02-08 07:24:01 -  WARNING - astropy - [logger.py:235 - _showwarning] - FITSFixedWarning: TSTART = / null value 
a floating-point value was expected.
2025-02-08 07:24:01 -  WARNING - astropy - [logger.py:235 - _showwarning] - FITSFixedWarning: TSTOP = / null value 
a floating-point value was expected.
2025-02-08 07:24:01 -  WARNING - astropy - [logger.py:235 - _showwarning] - FITSFixedWarning: TSTOP = / null value 
a floating-point value was expected.
2025-02-08 07:24:01 -  WARNING - astropy - [logger.py:235 - _showwarning] - FITSFixedWarning: RADECSYS= 'FK5 ' 
the RADECSYS keyword is deprecated, use RADESYSa.
2025-02-08 07:24:01 

a floating-point value was expected. [astropy.wcs.wcs]
a floating-point value was expected. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
Set MJD-OBS to 60661.709230 from DATE-OBS.
Set MJD-END to 60661.710388 from DATE-END'. [astropy.wcs.wcs]
this form of the PCi_ja keyword is deprecated, use PCi_ja. [astropy.wcs.wcs]
this form of the PCi_ja keyword is deprecated, use PCi_ja. [astropy.wcs.wcs]
this form of the PCi_ja keyword is deprecated, use PCi_ja. [astropy.wcs.wcs]
this form of the PCi_ja keyword is deprecated, use PCi_ja. [astropy.wcs.wcs]


In [9]:
ax1 = plt.subplot(1,2,1, projection=WCS(hdu1.header))
vmin, vmax = ZScaleInterval().get_limits(array)
ax1.imshow(array, origin='lower', vmin=vmin, vmax=vmax)
ax1.coords['ra'].set_axislabel('Right Ascension')
ax1.coords['dec'].set_axislabel('Declination')

ax2 = plt.subplot(1,2,2, projection=WCS(hdu1.header))
vmin, vmax = ZScaleInterval().get_limits(hdu1.data)
ax2.imshow(hdu1.data, origin='lower', vmin=vmin, vmax=vmax)
ax2.coords['ra'].set_axislabel('Right Ascension')
ax2.coords['dec'].set_axislabel('Declination')
ax2.coords['dec'].set_axislabel_position('r')
ax2.coords['dec'].set_ticklabel_position('r')

In [ ]:

frame_ps = mp.add_frame(image_data=hdu.data, projection=WCS(hdu.header), name="PS1DR2")

2025-02-04 19:40:08 -     INFO - celexta.main_plot - [main_plot.py: 89 -  set_focus] - Setting focus to frame: <celexta.frames.ImageFrame object at 0x13f1dcff0>
2025-02-04 19:40:08 -     INFO - celexta.main_plot - [main_plot.py: 89 -  set_focus] - Setting focus to frame: <celexta.frames.ImageFrame object at 0x13f1dcff0>


In [70]:
frame_ps = mp.frames[2]
frame_ls = mp.frames[1]

In [18]:
frame = mp.frames[0]
wcs_ref = frame.ax.wcs
ax_ref = frame.ax  # Reference frame's axis


array([[ 84.19643957, -25.23657255],
       [ 84.03071218, -25.37767537]])

In [26]:

# Get the displayed RA/DEC extent from the reference frame
xlim, ylim = ax_ref.get_xlim(), ax_ref.get_ylim()
ra_dec_corners = wcs_ref.all_pix2world([[xlim[0], ylim[0]], [xlim[1], ylim[1]]], 0)
ra_dec_corners

array([[ 84.17534349, -25.3021775 ],
       [ 84.16243629, -25.31317504]])

In [27]:
target_frame = mp.frames[1]
# Get the WCS of the target frame
wcs_target = target_frame.ax.wcs

# Convert world coordinates to pixel coordinates in the target frame
target_pixels = wcs_target.all_world2pix(ra_dec_corners, 0)
target_pixels

array([[405.27267684, 956.43571501],
       [565.61535387, 805.33836348]])

In [28]:
# Set the axis limits in the target frame to match the calculated pixel extent
target_frame.ax.set_xlim(target_pixels[:, 0])
target_frame.ax.set_ylim(target_pixels[:, 1])
mp.canvas.draw()

## New method with rotation

In [33]:
# Get the pixel limits of the reference frame
xlim_ref, ylim_ref = ax_ref.get_xlim(), ax_ref.get_ylim()

# Get the four corner pixel coordinates
corners_pix = np.array([
    [xlim_ref[0], ylim_ref[0]],  # Bottom-left
    [xlim_ref[0], ylim_ref[1]],  # Top-left
    [xlim_ref[1], ylim_ref[0]],  # Bottom-right
    [xlim_ref[1], ylim_ref[1]]   # Top-right
])

# Convert pixel coordinates to world coordinates (RA/Dec)
corners_world = wcs_ref.all_pix2world(corners_pix, 0)
corners_world

array([[ 84.18350946, -25.29154588],
       [ 84.15232072, -25.29071093],
       [ 84.18258941, -25.31974586],
       [ 84.15139279, -25.31891063]])

In [34]:
# Convert world coordinates to pixel coordinates in the target frame
corners_pix_target = wcs_target.all_world2pix(corners_world, 0)
corners_pix_target

array([[ 303.80407183, 1102.50235469],
       [ 691.27398687, 1114.00565031],
       [ 315.3073355 ,  715.0239452 ],
       [ 702.78492417,  726.52821996]])

In [58]:
overlay = ax1.get_coords_overlay('galactic')
overlay.grid(color='b', ls='dotted')
mp.canvas.draw()

In [37]:
# Get new x/y pixel limits from the transformed coordinates
xlim_target = [corners_pix_target[0, 0], corners_pix_target[1, 0]]
ylim_target = [np.min(corners_pix_target[:, 1]), np.max(corners_pix_target[:, 1])]

# Apply the new limits to the target frame
target_frame.ax.set_xlim(xlim_target)
target_frame.ax.set_ylim(ylim_target)
mp.canvas.draw()


In [111]:
hdu = get_vt_img()
wcs = WCS(hdu.header)
fig = plt.figure()
ax = fig.add_subplot(111, projection=wcs)
vmin, vmax = ZScaleInterval().get_limits(hdu.data)

ax.imshow(hdu.data, origin='lower', cmap='viridis', vmin=vmin, vmax=vmax)

2025-02-06 17:46:15 -    DEBUG - celexta.examples - [examples.py: 83 - get_vt_img] - Getting VT image
2025-02-06 17:46:15 -    DEBUG - celexta.examples - [examples.py: 83 - get_vt_img] - Getting VT image


### This method doesn't work
It fills aligned data with NaN

In [117]:
fig = plt.figure()

# Create a new WCS with North up (align to a standard celestial projection)
new_wcs = wcs.deepcopy()
new_wcs.wcs.pc = [[1, 0], [0, 1]]  # Identity matrix aligns North to up

# Reproject the image to the new aligned WCS
aligned_data, footprint = reproject_interp(hdu, new_wcs)

ax = fig.add_subplot(111, projection=new_wcs)

ax.imshow(aligned_data, origin='lower', cmap='viridis', vmin=vmin, vmax=vmax)

2025-02-06 17:50:25 -     INFO - reproject.common - [common.py:167 - _reproject_dispatcher] - Calling _reproject_full in non-dask mode
2025-02-06 17:50:25 -     INFO - reproject.common - [common.py:167 - _reproject_dispatcher] - Calling _reproject_full in non-dask mode


### This methods works but not sure the image is rotated in the right way

In [119]:
cd_matrix = wcs.wcs.cd
cd_matrix

array([[-6.32599186e-06, -2.11957101e-04],
       [-2.11951679e-04,  6.31373352e-06]])

In [137]:
det_cd = np.linalg.det(cd_matrix)
print(f"CD Matrix Determinant: {det_cd:.2e}")

CD Matrix Determinant: -4.50e-08


In [141]:

# Compute the position angle (North offset)
theta = np.arctan2(cd_matrix[1, 0], cd_matrix[0, 0]) * 180 / np.pi  # Convert to degrees
theta2 = np.arctan(cd_matrix[1, 0]/cd_matrix[0, 0]) * 180 / np.pi  # Convert to degrees

theta, theta2

(np.float64(-91.70956450551841), np.float64(88.29043549448159))

In [122]:
from scipy.ndimage import rotate
# Rotate the image to correct North alignment
aligned_data = rotate(hdu.data, -theta, reshape=False)

# Update WCS (reset rotation)
wcs.wcs.pc = [[1, 0], [0, 1]]  # Identity matrix

In [123]:
fig = plt.figure()

ax = fig.add_subplot(111, projection=wcs)

ax.imshow(aligned_data, origin='lower', cmap='viridis', vmin=vmin, vmax=vmax)

### Function to rotate image

In [10]:
def align_image_north(hdu):
    # Get the WCS
    wcs = WCS(hdu.header)

    # Compute the position angle (North offset)
    cd_matrix = wcs.wcs.cd
    theta = np.arctan2(cd_matrix[1, 0], cd_matrix[0, 0]) * 180 / np.pi  # Convert to degrees

    # Rotate the image to correct North alignment
    aligned_data = rotate(hdu.data, theta-180, reshape=False)

    # Update WCS (reset rotation)
    wcs.wcs.pc = [[1, 0], [0, 1]]  # Identity matrix

    return aligned_data, wcs

def align_image_north2(hdu):
    # Get the WCS
    wcs = WCS(hdu.header)

    # Compute the position angle (North offset)
    cd_matrix = wcs.wcs.cd
    theta = np.arctan2(cd_matrix[1, 0], cd_matrix[0, 0]) * 180 / np.pi  # Convert to degrees

    # Rotate the image to correct North alignment
    aligned_data = rotate(hdu.data, -theta, reshape=False)

    # Update WCS (reset rotation)
    wcs.wcs.pc = [[1, 0], [0, 1]]  # Identity matrix

    return aligned_data, wcs

## get rotation angle

In [17]:
import numpy as np

def extract_rotation_angle(R):
    """Extract the rotation angle theta from a 2D rotation matrix."""
    theta = np.arctan2(R[1, 0], R[0, 0])  # arctan2(sinθ, cosθ)
    return np.degrees(theta)  # Convert to degrees

# Example Rotation Matrix for 45° (counterclockwise)
R = np.array([
    [np.cos(np.radians(45)), -np.sin(np.radians(45))],
    [np.sin(np.radians(45)),  np.cos(np.radians(45))]
])

theta = extract_rotation_angle(R)
print(f"{R}, Rotation Angle: {theta:.2f}°")  # Output: 45.00°

[[ 0.70710678 -0.70710678]
 [ 0.70710678  0.70710678]], Rotation Angle: 45.00°


In [18]:
cd_matrix

array([[-6.32599186e-06, -2.11957101e-04],
       [-2.11951679e-04,  6.31373352e-06]])

In [24]:
import numpy as np
from astropy.io import fits

def get_north_east_angles(header):
    """Extract the North and East position angles from a FITS CD matrix.
    
    This returns the angle between the North and +X axis.
    """

    # Extract CD matrix values (or use PC if CD is not available)
    cd_matrix = np.array([
        [header.get("CD1_1", 1), header.get("CD1_2", 0)],
        [header.get("CD2_1", 0), header.get("CD2_2", 1)]
    ])
    
    # Compute angles
    theta_north = np.arctan2(cd_matrix[1,1], cd_matrix[0,1]) * 180 / np.pi  # North PA
    theta_east = np.arctan2(cd_matrix[1,0], cd_matrix[0,0]) * 180 / np.pi  # East PA
    theta_east = (theta_east + 180) % 360  # Ensure result is between 0-360°

    return theta_north, theta_east

# Example usage
theta_N, theta_E = get_north_east_angles(hdu_vt.header)
print(f"North PA: {theta_N:.2f}°")
print(f"East PA: {theta_E:.2f}°")  # Should be 90° offset from North

North PA: 178.29°
East PA: 88.29°


In [37]:
def get_rotation_angle(wcs):
    """Extract the rotation angle from a FITS CD matrix.
    
    This returns the angle between the North and +X axis.
    It can be used to align the image North
    by applying a counterclockwise rotation of the returned angle.
    """
    # Extract rotation matrix values (or use PC if CD is not available)
    if hasattr(wcs.wcs, "cd"):
        rot_matrix = wcs.wcs.cd
    elif hasattr(wcs.wcs, "pc"):
        rot_matrix = wcs.wcs.pc
    else:
        raise ValueError("Could not find rotation matrix in wcs")
    print(rot_matrix)
    print(np.linalg.det(rot_matrix))
    # Compute angle
    theta = np.arctan2(rot_matrix[1,0], rot_matrix[0,0]) * 180 / np.pi
    theta = (theta + 180) % 360  # Ensure result is between 0-360°

    return theta
for hdu in (hdu_ps1, hdu_vt, hdu_ls, hdu_qimb0, hdu_qimr0):
    print(hdu.name)
    with warnings.catch_warnings(): 
        warnings.simplefilter("ignore")
        print(get_rotation_angle(WCS(hdu.header)))

PRIMARY
[[-1.  0.]
 [ 0.  1.]]
-1.0
0.0
PRIMARY
[[-6.32599186e-06 -2.11957101e-04]
 [-2.11951679e-04  6.31373352e-06]]
-4.4964603987898585e-08
88.29043549448159
PRIMARY
[[-7.27777778e-05  0.00000000e+00]
 [ 0.00000000e+00  7.27777778e-05]]
-5.2966049382716104e-09
0.0
QIM1B_B0
[[-0.00018527 -0.00010294]
 [ 0.00010294 -0.0001853 ]]
4.4928520389463755e-08
330.9425136306485
QIM1B_R0
[[-0.00018543  0.00010288]
 [ 0.00010281  0.00018538]]
-4.495142155904157e-08
330.99264589834786


In [42]:
wcs.wcs.cd[1,1] = -wcs.wcs.cd[1,1]
wcs.wcs.cd

array([[-0.00018527, -0.00010294],
       [ 0.00010294,  0.0001853 ]])

In [36]:
hdu =fits.open( "~/Desktop/qimb0.fits")
hdu.info()

Filename: /Users/jp279903/Desktop/qimb0.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       4   ()      
  1  QIM1B_B0      1 ImageHDU        71   (461, 461)   int16   


In [34]:
hdu_qimr0.writeto("~/Desktop/qimr0.fits")

In [27]:
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    for hdu in (hdu_ps1, hdu_vt, hdu_ls, hdu_qimb0, hdu_qimr0):

        print(hdu.name)
        print(WCS(hdu))

PRIMARY
WCS Keywords

Number of WCS axes: 2
CTYPE : 'RA---TAN' 'DEC--TAN' 
CRVAL : np.float64(83.4146347045972) np.float64(-25.9999999999992) 
CRPIX : np.float64(10329.5) np.float64(-9018.5) 
PC1_1 PC1_2  : np.float64(-1.0) np.float64(0.0) 
PC2_1 PC2_2  : np.float64(0.0) np.float64(1.0) 
CDELT : np.float64(6.94444461259988e-05) np.float64(6.94444461259988e-05) 
NAXIS : 1320  1320
PRIMARY
WCS Keywords

Number of WCS axes: 2
CTYPE : 'RA---TAN' 'DEC--TAN' 
CRVAL : np.float64(84.1409802420383) np.float64(-25.3339676199326) 
CRPIX : np.float64(1064.09501787886) np.float64(1093.08677603824) 
CD1_1 CD1_2  : np.float64(-6.3259918584161e-06) np.float64(-0.00021195710077099) 
CD2_1 CD2_2  : np.float64(-0.00021195167888991) np.float64(6.31373352091863e-06) 
NAXIS : 2048  2048
PRIMARY
WCS Keywords

Number of WCS axes: 2
CTYPE : 'RA---TAN' 'DEC--TAN' 
CRVAL : np.float64(113.5004) np.float64(32.3762670333333) 
CRPIX : np.float64(504.0) np.float64(504.0) 
CD1_1 CD1_2  : np.float64(-7.27777777777778e-

In [70]:
# header = hdu2.header
wcs = WCS(header)
phi = header.get("PHI")
90 - phi

-88.28910462482438

In [97]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.wcs import WCS
from reproject import reproject_interp
from scipy.ndimage import rotate

def visualize_alignment(hdu, zscale=True):
    """Load a FITS image, align it so North is up, and visualize before/after."""
    
    original_data = hdu.data
    original_wcs = WCS(hdu)
    hdu2 = rotate_wcs(hdu, 45)
    aligned_data = hdu2.data
    aligned_wcs = WCS(hdu2)
    # Plot original and aligned images
    if zscale:
        vmin, vmax = ZScaleInterval().get_limits(original_data)
    else:
        vmin = 0
        vmax=1
    fig = plt.figure(figsize=(10, 5), tight_layout=True)
    ax1 = fig.add_subplot(1, 2, 1, projection=original_wcs)
    # Original image
    ax1.imshow(original_data, origin="lower", cmap="gray", 
               vmin=vmin, vmax=vmax,
               )
    ax1.set_title("Original Image (unrotated)")

    ax2 = fig.add_subplot(1, 2, 2, projection=aligned_wcs)
    # Manually rotated image
    ax2.imshow(aligned_data, origin="lower", cmap="gray", 
               vmin=vmin, vmax=vmax,
               )
    ax2.set_title(f"Manually Rotated ({theta:.2f}°)")

    # Reprojected image
    # axes[2].imshow(reprojected_data, origin="lower", cmap="gray", vmin=vmin, vmax=vmax)
    # axes[2].set_title("Reprojected (North Up)")

    plt.show()
hdu_ls = fits.open("~/Desktop/ls_img.fits")[0]
hdu_qim = fits.open("~/.celexta/cache/svom/sb25020504/qim1b_vt.fits")[3]
if hdu_qim.name.split("_")[-1][0] == "R":
    hdu_qim.data = hdu_qim.data.T
visualize_alignment(hdu2,
                    #  zscale=False,
                     )

2025-02-08 10:08:44 -  WARNING - astropy - [logger.py:235 - _showwarning] - FITSFixedWarning: TSTART = / null value 
a floating-point value was expected.


2025-02-08 10:08:44 -  WARNING - astropy - [logger.py:235 - _showwarning] - FITSFixedWarning: TSTART = / null value 
a floating-point value was expected.
2025-02-08 10:08:44 -  WARNING - astropy - [logger.py:235 - _showwarning] - FITSFixedWarning: TSTOP = / null value 
a floating-point value was expected.
2025-02-08 10:08:44 -  WARNING - astropy - [logger.py:235 - _showwarning] - FITSFixedWarning: TSTOP = / null value 
a floating-point value was expected.
2025-02-08 10:08:45 -  WARNING - astropy - [logger.py:235 - _showwarning] - FITSFixedWarning: RADECSYS= 'FK5 ' 
the RADECSYS keyword is deprecated, use RADESYSa.
2025-02-08 10:08:45 -  WARNING - astropy - [logger.py:235 - _showwarning] - FITSFixedWarning: RADECSYS= 'FK5 ' 
the RADECSYS keyword is deprecated, use RADESYSa.
2025-02-08 10:08:45 -  WARNING - astropy - [logger.py:235 - _showwarning] - FITSFixedWarning: 'datfix' made the change 'Set DATEREF to '2017-01-01' from MJDREF.
Set MJD-OBS to 60661.709230 from DATE-OBS.
Set MJD-END 

a floating-point value was expected. [astropy.wcs.wcs]
a floating-point value was expected. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
Set MJD-OBS to 60661.709230 from DATE-OBS.
Set MJD-END to 60661.710388 from DATE-END'. [astropy.wcs.wcs]


: 

In [ ]:
hdul_qim = fits.open("~/.celexta/cache/svom/sb25020504/qim1b_vt.fits")
hdul_qim.info()
ls_img = fits.open("/Users/jp279903/.zhunter/cache/download/url/f6d9c0df2702dd2242b7ea63239ce493/contents")
ls_img.info()

Filename: /Users/jp279903/.celexta/cache/svom/sb25020504/qim1b_vt.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PrimaryHDU    1 PrimaryHDU     107   ()      
  1  QIM1B_B0      1 ImageHDU        71   (461, 461)   int16   
  2  QIM1B_B1      1 ImageHDU        71   (461, 461)   int16   
  3  QIM1B_R0      1 ImageHDU        71   (461, 461)   int16   
  4  QIM1B_R1      1 ImageHDU        71   (461, 461)   int16   


## Rotate WCS AND data

In [95]:
def align_image_north_old(hdu):
    # Load the image and WCS
    wcs = WCS(hdu.header)
    original_data = hdu.data
    # angle between +X and N?
    phi = hdu.header.get("PHI", None)
    if phi is not None:
        log.debug("Using PHI information")
        # Add 90 degrees because we align +Y to north
        theta = 90-phi
    else:
        # Compute rotation angle from WCS CD matrix
        if hasattr(wcs.wcs, "cd"):
            log.debug("Using CD matrix")
            rot_matrix = wcs.wcs.cd
        else:
            log.debug("Using PC matrix")
            rot_matrix = wcs.wcs.pc

        theta = np.arctan2(rot_matrix[1, 0], rot_matrix[0, 0]) * 180 / np.pi  # Convert to degrees

    if theta == 0:
        aligned_data = original_data
    else:
        # Rotate image counterclockwise (as defined by FITS standard)
        aligned_data = rotate(original_data, -theta, reshape=False)

    return aligned_data

def rotate_wcs(hdu, theta_deg):
    """Rotate the WCS of a FITS image by theta degrees."""
    # Load the FITS file
    wcs = WCS(hdu.header)

    # Convert angle to radians
    theta_rad = np.radians(theta_deg)

    # Rotation matrix
    R = np.array([
        [np.cos(theta_rad), -np.sin(theta_rad)],
        [np.sin(theta_rad),  np.cos(theta_rad)]
    ])

    # Extract the existing CD (or PC) matrix
    cd_matrix = np.array([
        [wcs.wcs.cd[0,0], wcs.wcs.cd[0,1]],
        [wcs.wcs.cd[1,0], wcs.wcs.cd[1,1]]
    ])

    # Apply rotation: new_CD = R * old_CD
    new_cd_matrix = np.dot(R, cd_matrix)

    # Update WCS with the new CD matrix
    wcs.wcs.cd = new_cd_matrix

    # Compute the expanded shape to fit the full rotated image
    h, w = hdu.data.shape
    diag = int(np.ceil(np.sqrt(h**2 + w**2)))  # Diagonal size to fit full rotation
    new_shape = (diag, diag)

    # Compute center shift to preserve the same center
    old_center = np.array([w / 2, h / 2])  # Original center in pixels
    new_center = np.array([diag / 2, diag / 2])  # New center in pixels

    # Adjust WCS reference pixel to match new center
    shift = new_center - old_center
    wcs.wcs.crpix += shift  # Shift CRPIX to maintain alignment

    # Reproject the image with the expanded shape
    reprojected_data, _ = reproject_interp(hdu, wcs, shape_out=new_shape)
    # # Update data
    # reprojected_data = reproject_interp(hdu, wcs, 
    #                                        return_footprint=False,
    #                                     #    shape_out=hdu.data.shape,
    #                                     )
    # Save the rotated WCS back to the FITS header
    hdu2 = hdu.copy()
    hdu2.data = reprojected_data
    hdu2.header.update(wcs.to_header())

    print(f"Rotated WCS and reprojected image by {theta_deg}°")
    return hdu2

In [28]:
def say_hello(event):
    print(f"Hello ! {event}")
    art = event.artist
    print("I am an artist", art)
    art.set_linestyle("dashed")
    ind = event.ind[0]  # Get the index of the clicked point
    colors = scatter.get_facecolors()
    colors[ind] = [1, 0, 0, 1]  # Change color to red
    scatter.set_edgecolors('black')  # Ensure all points have visible edges
    scatter.set_sizes([200 if i == ind else 100 for i in range(len(x))])  # Enlarge selected point
    fig.canvas.draw()


In [32]:
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
cv = FigureCanvas()

In [29]:
import matplotlib.pyplot as plt
import numpy as np
fig, ax = plt.subplots()
scatter = ax.scatter(tab["RA"], tab["DEC"], picker=True, facecolor="none", edgecolor="black")
fig.canvas.mpl_connect("pick_event", say_hello)

14

Hello ! <matplotlib.backend_bases.PickEvent object at 0x303cd8a40>
I am an artist <matplotlib.collections.PathCollection object at 0x303cd4290>


Traceback (most recent call last):
  File "/opt/miniconda3/envs/celexta_dev/lib/python3.12/site-packages/matplotlib/cbook.py", line 298, in process
    func(*args, **kwargs)
  File "/var/folders/w0/p4yn239n5zn2zbpmyn8hbtcjh2rltm/T/ipykernel_51102/2396062776.py", line 8, in say_hello
    colors[ind] = [1, 0, 0, 1]  # Change color to red
    ~~~~~~^^^^^
IndexError: index 0 is out of bounds for axis 0 with size 0


In [17]:
# scatter.set_edgecolors("red")
scatter.set_visible(False)


In [30]:
toolbar

NameError: name 'toolbar' is not defined

In [21]:
np.random.choice([SphericalCircle,2])

astropy.visualization.wcsaxes.patches.SphericalCircle

# Test ImageFrame with MWE

## Maptlotlib

In [3]:
from celexta.frames import ImageFrame
from PyQt6.QtWidgets import QMainWindow
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas

parent = QMainWindow()
parent.canvas = FigureCanvas()
parent.setCentralWidget(parent.canvas)
parent.show()
frame = ImageFrame(parent)

In [4]:

im_data, wcs = generate_example_image()
frame.display_image(im_data, wcs)


2025-02-08 07:19:18 -    DEBUG - celexta.examples - [examples.py: 93 - get_vt_img] - Getting VT image
2025-02-08 07:19:18 -    DEBUG - celexta.examples - [examples.py: 93 - get_vt_img] - Getting VT image


In [27]:
frame.img_display.get_extent()

[-0.5, 2047.5, -0.5, 2047.5]

In [32]:
frame.ax.wcs.header

AttributeError: 'WCS' object has no attribute 'header'

In [30]:
xmin, xmax, ymin, ymax = frame.img_display.get_extent()
x_cen = (xmax+xmin)/2
y_cen = (ymax+ymin)/2
center = wcs.pixel_to_world(x_cen, y_cen)
center


<SkyCoord (FK5: equinox=2000.0): (ra, dec) in deg
    (84.15733348, -25.32600864)>

In [10]:
frame.update_contrast_and_brightness(contrast=0.5, brightness=0.5)


In [5]:
from celexta.regions import CircleRegion
from astropy.coordinates import SkyCoord

region = CircleRegion(SkyCoord(84.1409802420383, -25.3339676199326, unit="deg"), 1 * u.arcmin)
frame.add_region(region)


2025-02-02 19:36:47 -    DEBUG - celexta.mpl_widget - [mpl_widget.py:720 - _add_region] - Adding region to frame.


In [6]:
frame.remove_region(region)

2025-02-02 19:28:53 -    DEBUG - celexta.mpl_widget - [mpl_widget.py:746 - remove_region] - Removing region from frame.


In [7]:
frame.add_region(region)


2025-02-02 19:28:53 -    DEBUG - celexta.mpl_widget - [mpl_widget.py:715 - _add_region] - Adding region to frame.


In [6]:
region.update_aesthetics(color="red")

2025-02-02 19:36:53 -    DEBUG - celexta.mpl_widget - [mpl_widget.py:770 - update_region_aesthetics] - Updating region aesthetics.


In [7]:
region.update_position(center=SkyCoord(84.189, -25.309, unit="deg"), radius=2 * u.arcmin)


2025-02-02 19:36:56 -    DEBUG - celexta.mpl_widget - [mpl_widget.py:759 - update_region_position] - Updating region position.
2025-02-02 19:36:56 -    DEBUG - celexta.mpl_widget - [mpl_widget.py:720 - _add_region] - Adding region to frame.


In [10]:
frame.highlight_region(region)


In [11]:
from zhunter.catalogs import get_ls_image
im = get_ls_image(ra=84.189, dec=-25.309, size=1024)

2025-02-02 19:39:35 -     INFO - zhunter.catalogs - [catalogs.py: 21 - get_ls_image] - Trying to fetch r band image of size 1024 pixels from LSDR10


In [15]:
frame.display_image(im.data, WCS(im))


## Pyqtgraph

In [35]:
cb = pg.ColorBarItem()
cb

In [39]:
cb.colorMap()

In [40]:
hlw = pg.HistogramLUTWidget()
hlw

In [ ]:
for tick in hlw.item.gradient.listTicks():
    tick.hide()


[]

In [45]:
hlw.item.gradient.setColorMap(cb.colorMap())

AttributeError: 'NoneType' object has no attribute 'getColors'

In [21]:
from astropy.io import fits
hdul = fits.open('/Users/jp279903/.celexta/config/latest_session_data/Examples/vt_frame_img.fits')
hdul.info()


Filename: /Users/jp279903/.celexta/config/latest_session_data/Examples/vt_frame_img.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      96   (2048, 2048)   float64   


In [24]:
import pyqtgraph as pg
import numpy as np
im = pg.ImageItem()
im.setImage(np.ones((100, 100)))

In [ ]:
from astropy.table import Table
fname = "/Users/jp279903/.celexta/config/latest_session_data/Examples/Test Candidate.ecsv"
tab = Table.read(fname)

In [32]:
im.getColorMap().name

'viridis'

In [4]:
hdul = fits.open("/Users/jp279903/SVOM_pipeline/vtac/tests/data/QCANDI_VT.fits")
hdul.info()


Filename: /Users/jp279903/SVOM_pipeline/vtac/tests/data/QCANDI_VT.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PrimaryHDU    1 PrimaryHDU      46   ()      
  1  VT_ID_5       1 BinTableHDU     40   4R x 7C   [23A, D, D, D, D, D, D]   
  2  VT_ID_99      1 BinTableHDU     40   4R x 7C   [23A, D, D, D, D, D, D]   
  3  VT_ID_103     1 BinTableHDU     40   4R x 7C   [23A, D, D, D, D, D, D]   


In [28]:
hdul[0].header

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                    8 / array data type                                
NAXIS   =                    0 / number of array dimensions                     
EXTEND  =                    T                                                  
EXTNAME = 'PrimaryHDU'                                                          
ORIGIN  = 'FSC     '           / Science Center                                 
TELESCOP= 'SVOM    '           / Official acronym for the mission               
INSTRUME= 'VT      '           / Instrument acronym                             
CREATOR = 'VTAC    '           / Pipeline that created or modified this file    
CARD    = 'QCANDI_VT'          / Product card                                   
PROGRAM = 'CoreProgram'        / Mission program                                
PRDM_VER=                 1.12 / Product Data Model Version                     
GLDM_VER=  / Global Data Mod

In [20]:
pi = pg.PlotWidget()
pi.show()
for row in hdul[1:]:
    ra = row.header["RA"]
    dec = row.header["DEC"]
    vt_id = row.header["VT_ID"]
    _tab = Table.read(row)
    print(vt_id, ra,dec,_tab)
    obs = []
    for _obs in _tab:
        for band in ("R", "B"):
            if np.isfinite(_obs[f"MAG_{band}"]):
                vt_obs = PhotometricPoint(
                        mag=_obs[f"MAG_{band}"] * u.ABmag,
                        unc=_obs[f"MAG_{band}_ERR"] * u.mag if _obs[f"MAG_{band}_ERR"] > 0 else 0.01 * u.mag, # 0.01 mag minimum uncertainty
                        phot_filter=f"VT_{band}",
                                                # phot_filter=f"VT_{band}",
                        obs_time=_obs["DATE-OBS"],
                        obs_duration=300 * u.s,
                    )
            else:
                vt_obs = PhotometricPoint(
                        mag=_obs[f"MAG_{band}_LIM"] * u.ABmag,
                        unc=0.0 * u.mag,
                        limit=True,
                        phot_filter=f"VT_{band}",
                                                # phot_filter=f"VT_{band}",
                        obs_time=_obs["DATE-OBS"],
                        obs_duration=300 * u.s,
                    )
            obs.append(vt_obs)
                
    for o in obs:
        o.plot_pyqt(vb=pi, t0=Time(hdul[0].header["TT_ECL"])-4*u.d, mode="temporal", xlogscale=True)


5 212.6389636 29.6993735         DATE-OBS            MAG_R      MAG_R_ERR   ...  MAG_B_ERR   MAG_B_LIM
----------------------- ------------- ------------ ... ------------ ---------
2023-10-13T09:12:46.000 17.1300001526          0.0 ...          0.0     23.01
2023-10-13T09:17:46.000 18.0200014496 0.0099999998 ... 0.0099999998      23.4
2023-10-13T10:46:46.000 20.2600012207 0.0099999998 ... 0.0099999998     23.41
2023-10-13T11:21:46.000 20.5800009155 0.0199999996 ... 0.0199999996     23.46
2025-02-25 09:10:03 -    DEBUG - zhunter.photometry - [photometry.py:342 - create_visual_representation] - Creating visual representation of photometric data point
2025-02-25 09:10:03 -    DEBUG - zhunter.photometry - [photometry.py:438 - create_errorbar] - Creating error bar for photometric data point
2025-02-25 09:10:03 -     INFO - zhunter.photometry - [photometry.py:462 - create_errorbar] - Making temporal plot relative to t0: 2023-10-13T00:00:53.709 (displayed in s)
2025-02-25 09:10:03 -     INFO 

In [27]:
from zhunter.photometry import PhotometricFilter
phf = PhotometricFilter("VT_B")
print(phf)
pi = pg.PlotWidget()
pi.plot(phf.wavelength, phf.transmission)
pi.show()

PhotometricFilter: VT_B (center=5322.247 Angstrom, width=2800.000 Angstrom)


In [ ]:
pi = pg.PlotWidget()
for o in obs:
    o.plot_pyqt(vb=pi, t0=Time(hdul[0].header["TT_ECL"])-4*u.d, mode="temporal", xlogscale=True)
pi.show()

2025-02-25 07:27:10 -    DEBUG - zhunter.photometry - [photometry.py:342 - create_visual_representation] - Creating visual representation of photometric data point
2025-02-25 07:27:10 -    DEBUG - zhunter.photometry - [photometry.py:438 - create_errorbar] - Creating error bar for photometric data point
2025-02-25 07:27:10 -     INFO - zhunter.photometry - [photometry.py:462 - create_errorbar] - Making temporal plot relative to t0: 2023-10-13T00:00:53.709 (displayed in s)
2025-02-25 07:27:10 -     INFO - zhunter.photometry - [photometry.py:480 - create_errorbar] - Converting x axis to logscale because t0 is present
2025-02-25 07:27:10 -    DEBUG - zhunter.photometry - [photometry.py:486 - create_errorbar] - x, xerr = 4.522e+00, [[0.00196293]
 [0.0019541 ]]
2025-02-25 07:27:10 -    DEBUG - zhunter.conversions - [conversions.py: 80 - convert_flux_with_unc_propagation] - Both units are the same (mag(AB)), no conversion needed.
2025-02-25 07:27:10 -    DEBUG - zhunter.conversions - [convers

2025-02-25 07:27:10 -     INFO - zhunter.photometry - [photometry.py:462 - create_errorbar] - Making temporal plot relative to t0: 2023-10-13T00:00:53.709 (displayed in s)
2025-02-25 07:27:10 -     INFO - zhunter.photometry - [photometry.py:480 - create_errorbar] - Converting x axis to logscale because t0 is present
2025-02-25 07:27:10 -    DEBUG - zhunter.photometry - [photometry.py:486 - create_errorbar] - x, xerr = 4.522e+00, [[0.00196293]
 [0.0019541 ]]
2025-02-25 07:27:10 -    DEBUG - zhunter.conversions - [conversions.py: 80 - convert_flux_with_unc_propagation] - Both units are the same (mag(AB)), no conversion needed.
2025-02-25 07:27:10 -    DEBUG - zhunter.conversions - [conversions.py: 89 - convert_flux_with_unc_propagation] - Returning y, yp, ym = 17.5200009155 mag(AB), 0.01 mag, 0.01 mag
2025-02-25 07:27:10 -    DEBUG - zhunter.photometry - [photometry.py:506 - create_errorbar] - y, yerr = 1.752e+01, [[0.01]
 [0.01]]
2025-02-25 07:27:10 -    DEBUG - zhunter.photometry - [ph

The cached device pixel ratio value was stale on window expose.  Please file a QTBUG which explains how to reproduce.
The cached device pixel ratio value was stale on window expose.  Please file a QTBUG which explains how to reproduce.
The cached device pixel ratio value was stale on window expose.  Please file a QTBUG which explains how to reproduce.
The cached device pixel ratio value was stale on window expose.  Please file a QTBUG which explains how to reproduce.
The cached device pixel ratio value was stale on window expose.  Please file a QTBUG which explains how to reproduce.
The cached device pixel ratio value was stale on window expose.  Please file a QTBUG which explains how to reproduce.
The cached device pixel ratio value was stale on window expose.  Please file a QTBUG which explains how to reproduce.


In [54]:
Table.read(hdul["CAT_LSDR10"])

VT_ID,RA_R0,RA_R1,RA_R2,RA_R3,RA_B0,RA_B1,RA_B2,RA_B3,DEC_R0,DEC_R1,DEC_R2,DEC_R3,DEC_B0,DEC_B1,DEC_B2,DEC_B3,MAG_R_R0,MAG_R_R1,MAG_R_R2,MAG_R_R3,MAG_R_B0,MAG_R_B1,MAG_R_B2,MAG_R_B3,MAG_R_ERR_R0,MAG_R_ERR_R1,MAG_R_ERR_R2,MAG_R_ERR_R3,MAG_R_ERR_B0,MAG_R_ERR_B1,MAG_R_ERR_B2,MAG_R_ERR_B3,MAG_B_R0,MAG_B_R1,MAG_B_R2,MAG_B_R3,MAG_B_B0,MAG_B_B1,MAG_B_B2,MAG_B_B3,MAG_B_ERR_R0,MAG_B_ERR_R1,MAG_B_ERR_R2,MAG_B_ERR_R3,MAG_B_ERR_B0,MAG_B_ERR_B1,MAG_B_ERR_B2,MAG_B_ERR_B3,ANGDIST_R0,ANGDIST_R1,ANGDIST_R2,ANGDIST_R3,ANGDIST_B0,ANGDIST_B1,ANGDIST_B2,ANGDIST_B3,DMAG_R0,DMAG_R1,DMAG_R2,DMAG_R3,DMAG_B0,DMAG_B1,DMAG_B2,DMAG_B3
,deg,deg,deg,deg,deg,deg,deg,deg,deg,deg,deg,deg,deg,deg,deg,deg,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,arcsec,arcsec,arcsec,arcsec,arcsec,arcsec,arcsec,arcsec,,,,,,,,
int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
1,--,--,--,--,113.54836882300768,113.54836882300768,113.54836882300768,0.0,--,--,--,--,32.36728210433652,32.36728210433652,32.36728210433652,0.0,--,--,--,--,13.604792594909668,13.604792594909668,13.604792594909668,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,--,--,--,--,13.891736030578612,13.891736030578612,13.891736030578612,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,--,--,--,--,0.035088,0.036329,0.089962,0.0,--,--,--,--,-0.32625198362138796,-0.2612514496213887,-0.21625232692138852,0.0
2,113.51417864970084,113.51417864970084,--,113.51417864970084,113.51417864970084,113.51417864970084,113.51417864970084,0.0,32.34779065737403,32.34779065737403,--,32.34779065737403,32.34779065737403,32.34779065737403,32.34779065737403,0.0,13.628472328186035,13.628472328186035,--,13.628472328186035,13.628472328186035,13.628472328186035,13.628472328186035,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.216758728027344,14.216758728027344,--,14.216758728027344,14.216758728027344,14.216758728027344,14.216758728027344,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.011171,0.017501,--,0.042372,0.070719,0.067628,0.085749,0.0,-0.024083137513965625,-0.04908275601396461,--,-0.169083595313964,-0.23122882847265558,-0.13622951507265668,-0.08122920987265658,0.0
3,--,--,--,--,113.48142263400987,113.48142263400987,113.48142263400987,0.0,--,--,--,--,32.39111469326149,32.39111469326149,32.39111469326149,0.0,--,--,--,--,14.438010215759276,14.438010215759276,14.438010215759276,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,--,--,--,--,15.570302963256836,15.570302963256836,15.570302963256836,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,--,--,--,--,0.380026,0.407747,0.70183,0.0,--,--,--,--,0.3723154067568366,0.44731426235683536,0.5123147964568364,0.0
4,--,--,--,--,113.55866830319296,113.55866830319296,113.55866830319296,0.0,--,--,--,--,32.38025319518838,32.38025319518838,32.38025319518838,0.0,--,--,--,--,15.676219940185549,15.676219940185549,15.676219940185549,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,--,--,--,--,16.210620880126953,16.210620880126953,16.210620880126953,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,--,--,--,--,0.059158,0.045312,0.158256,0.0,--,--,--,--,-0.16736602787304733,-0.08236694337304584,-0.03736877437304642,0.0
5,113.5266187554828,113.5266187554828,113.5266187554828,--,113.5266187554828,113.5266187554828,113.5266187554828,0.0,32.34636392627745,32.34636392627745,32.34636392627745,--,32.34636392627745,32.34636392627745,32.34636392627745,0.0,15.74032497406006,15.74032497406006,15.74032497406006,--,15.74032497406006,15.74032497406006,15.74032497406006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.24835205078125,16.24835205078125,16.24835205078125,--,16.24835205078125,16.24835205078125,16.24835205078125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.101543,0.096748,0.